In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
MONTHS = [6,7,8,9]
SPECIES = ['CULEX PIPIENS', 'CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS'] 
TRAPS = []
MAX_CATCH_DATE = False

In [3]:
import pandas as pd


train_df = pd.read_csv('../data/raw/train.csv')
test_df = pd.read_csv('../data/raw/test.csv')
weather_df = pd.read_csv('../data/raw/weather.csv')

In [4]:
from src.data import clean_weather

weather_df = clean_weather(weather_df)

In [5]:
from src.data import remove_address, split_date

train_df = remove_address(split_date(train_df))
test_df = remove_address(split_date(test_df))

In [6]:
train_df = train_df[train_df.Month.isin(MONTHS)]
train_df = train_df[train_df.Species.isin(SPECIES)]

test_df = test_df[test_df.Month.isin(MONTHS)]
test_df = test_df[test_df.Species.isin(SPECIES)]

In [4]:
# from src.data import preprocess_data

# data = preprocess_data({'train': train_df, 'test': test_df, 'weather': weather_df})

# train_df = data['train']
# test_df = data['test']
# weather_df = data['weather']

[Pipeline]  (step 1 of 3) Processing functiontransformer-1, total=   0.0s
[Pipeline]  (step 2 of 3) Processing monthspeciestraptransformer, total=   0.0s
[Pipeline]  (step 3 of 3) Processing functiontransformer-2, total=   0.0s


In [5]:
trap_max_mos = train_df.groupby(['Date', 'Trap'])['NumMosquitos'].sum().reset_index().groupby(['Trap'])['NumMosquitos'].max()
trap_wnv_proba = train_df.groupby(['Date', 'Trap'])['WnvPresent'].max().reset_index().groupby(['Trap'])['WnvPresent'].mean()

In [6]:
import numpy as np


trap_max_mos.corr(trap_wnv_proba)

np.float64(0.4300605114998707)

In [7]:
single_row = train_df.groupby(
    ['Date', 'Trap', 'Species']
).agg(
    RowCount=('NumMosquitos', 'count'),
    NumMosquitos=('NumMosquitos', 'sum'),
    WnvPresent=('WnvPresent', 'max')
).reset_index()

single_row

,Date,Trap,Species,RowCount,NumMosquitos,WnvPresent
0,2007-06-05,T002,CULEX PIPIENS,1,1,0
1,2007-06-05,T002,CULEX PIPIENS/RESTUANS,1,3,0
2,2007-06-05,T002,CULEX RESTUANS,1,5,0
3,2007-06-05,T009,CULEX RESTUANS,1,2,0
4,2007-06-05,T011,CULEX RESTUANS,1,1,0
...,...,...,...,...,...,...
8083,2013-09-26,T232,CULEX PIPIENS/RESTUANS,1,1,0
8084,2013-09-26,T233,CULEX PIPIENS/RESTUANS,1,5,0
8085,2013-09-26,T235,CULEX PIPIENS/RESTUANS,1,1,0
8086,2013-09-26,T900,CULEX PIPIENS,4,37,0


In [8]:
single_row.WnvPresent.mean()

np.float64(0.05650346191889219)

In [9]:
single_row.RowCount.max()

np.int64(51)

In [10]:
single_row.NumMosquitos.max()

np.int64(2532)

In [11]:
single_row[['RowCount', 'NumMosquitos']].corr()

,RowCount,NumMosquitos
RowCount,1.000000,0.962477
NumMosquitos,0.962477,1.000000


In [12]:
train_df.columns

Index(['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'NumMosquitos',
       'WnvPresent', 'Month', 'Year', 'Week', 'Dayofyear', 'Dayofweek'],
      dtype='object')

In [13]:
single_row.columns

Index(['Date', 'Trap', 'Species', 'RowCount', 'NumMosquitos', 'WnvPresent'], dtype='object')

In [14]:
train_df = single_row.merge(
    train_df[
        ['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'Month', 'Year', 'Week', 'Dayofyear', 'Dayofweek']
    ].drop_duplicates(),
    on=['Date', 'Species', 'Trap'],
    how='left'
)

In [15]:
train_df

,Date,Trap,Species,RowCount,NumMosquitos,WnvPresent,Latitude,Longitude,Month,Year,Week,Dayofyear,Dayofweek
0,2007-06-05,T002,CULEX PIPIENS,1,1,0,41.954690,-87.800991,6,2007,23,156,1
1,2007-06-05,T002,CULEX PIPIENS/RESTUANS,1,3,0,41.954690,-87.800991,6,2007,23,156,1
2,2007-06-05,T002,CULEX RESTUANS,1,5,0,41.954690,-87.800991,6,2007,23,156,1
3,2007-06-05,T009,CULEX RESTUANS,1,2,0,41.981964,-87.812827,6,2007,23,156,1
4,2007-06-05,T011,CULEX RESTUANS,1,1,0,41.944869,-87.832763,6,2007,23,156,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8083,2013-09-26,T232,CULEX PIPIENS/RESTUANS,1,1,0,41.912563,-87.668055,9,2013,39,269,3
8084,2013-09-26,T233,CULEX PIPIENS/RESTUANS,1,5,0,42.009876,-87.807277,9,2013,39,269,3
8085,2013-09-26,T235,CULEX PIPIENS/RESTUANS,1,1,0,41.776428,-87.627096,9,2013,39,269,3
8086,2013-09-26,T900,CULEX PIPIENS,4,37,0,41.974689,-87.890615,9,2013,39,269,3


In [16]:
test_df

,Id,Date,Species,Trap,Latitude,Longitude,Month,Year,Week,Dayofyear,Dayofweek
0,1,2008-06-11,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,6,2008,24,163,2
1,2,2008-06-11,CULEX RESTUANS,T002,41.954690,-87.800991,6,2008,24,163,2
2,3,2008-06-11,CULEX PIPIENS,T002,41.954690,-87.800991,6,2008,24,163,2
8,9,2008-06-11,CULEX PIPIENS/RESTUANS,T007,41.994991,-87.769279,6,2008,24,163,2
9,10,2008-06-11,CULEX RESTUANS,T007,41.994991,-87.769279,6,2008,24,163,2
...,...,...,...,...,...,...,...,...,...,...,...
116278,116279,2014-10-02,CULEX RESTUANS,T094B,41.719140,-87.669539,10,2014,40,275,3
116279,116280,2014-10-02,CULEX PIPIENS,T094B,41.719140,-87.669539,10,2014,40,275,3
116285,116286,2014-10-02,CULEX PIPIENS/RESTUANS,T054C,41.925652,-87.633590,10,2014,40,275,3
116286,116287,2014-10-02,CULEX RESTUANS,T054C,41.925652,-87.633590,10,2014,40,275,3


In [17]:
train_df.Trap.unique()

array(['T002', 'T009', 'T011', 'T015', 'T016', 'T019', 'T025', 'T028',
       'T031', 'T033', 'T045', 'T046', 'T048', 'T049', 'T050', 'T054',
       'T086', 'T089', 'T090', 'T092', 'T094', 'T096', 'T135', 'T141',
       'T142', 'T143', 'T145', 'T146', 'T147', 'T149', 'T150', 'T151',
       'T152', 'T153', 'T154', 'T158', 'T159', 'T162', 'T218', 'T220',
       'T001', 'T003', 'T006', 'T007', 'T008', 'T012', 'T034', 'T037',
       'T040', 'T043', 'T047', 'T051', 'T085', 'T088', 'T129', 'T148',
       'T161', 'T219', 'T013', 'T014', 'T018', 'T030', 'T084', 'T091',
       'T144', 'T160', 'T005', 'T017', 'T044', 'T095', 'T004', 'T035',
       'T036', 'T039', 'T060', 'T061', 'T062', 'T065', 'T066', 'T067',
       'T069', 'T070', 'T071', 'T073', 'T074', 'T076', 'T077', 'T079',
       'T080', 'T081', 'T082', 'T083', 'T114', 'T155', 'T063', 'T115',
       'T138', 'T200', 'T206', 'T209', 'T212', 'T215', 'T107', 'T128',
       'T072', 'T027', 'T075', 'T078', 'T097', 'T099', 'T100', 'T102',
      

In [18]:
len(train_df.Trap.str[:4].unique())

134

In [19]:
print(trap_max_mos['T094'], trap_max_mos['T094B'], trap_wnv_proba['T094'], trap_wnv_proba['T094B'])
print(trap_max_mos['T054'], trap_max_mos['T054C'], trap_wnv_proba['T054'], trap_wnv_proba['T054C'])

167 20 0.06779661016949153 0.0
129 60 0.06349206349206349 0.14285714285714285


In [20]:
test_df.Trap.unique()

array(['T002', 'T007', 'T015', 'T045', 'T046', 'T048', 'T049', 'T050',
       'T054', 'T086', 'T091', 'T094', 'T096', 'T129', 'T143', 'T148',
       'T153', 'T159', 'T009', 'T011', 'T016', 'T019', 'T025', 'T028',
       'T031', 'T033', 'T089', 'T090', 'T092', 'T135', 'T141', 'T142',
       'T145', 'T146', 'T147', 'T149', 'T150', 'T151', 'T152', 'T154',
       'T158', 'T162', 'T218', 'T220', 'T001', 'T003', 'T006', 'T008',
       'T012', 'T034', 'T037', 'T040', 'T043', 'T047', 'T051', 'T085',
       'T088', 'T161', 'T219', 'T013', 'T014', 'T018', 'T030', 'T084',
       'T144', 'T160', 'T005', 'T017', 'T044', 'T095', 'T004', 'T035',
       'T036', 'T039', 'T060', 'T061', 'T062', 'T065', 'T066', 'T067',
       'T069', 'T070', 'T071', 'T073', 'T074', 'T075', 'T076', 'T077',
       'T079', 'T080', 'T081', 'T082', 'T083', 'T114', 'T155', 'T063',
       'T115', 'T138', 'T200', 'T206', 'T209', 'T212', 'T215', 'T107',
       'T128', 'T072', 'T078', 'T097', 'T099', 'T100', 'T102', 'T103',
      

In [21]:
len(test_df.Trap.str[:4].unique())

135

In [22]:
test_df.Trap.str[:4].unique()[~pd.Series(test_df.Trap.str[:4].unique()).isin(train_df.Trap.str[:4].unique())]

array(['T234'], dtype=object)

In [23]:
train_df.Trap = train_df.Trap.str[:4]
test_df.Trap = test_df.Trap.str[:4]

In [24]:
train_df['trap_max_mos'] = train_df.Trap.map(trap_max_mos.to_dict())
train_df['trap_wnv_proba'] = train_df.Trap.map(trap_wnv_proba.to_dict())

# mean or median
test_df['trap_max_mos'] = test_df.Trap.map(trap_max_mos.to_dict()).fillna(trap_max_mos.median())
test_df['trap_wnv_proba'] = test_df.Trap.map(trap_wnv_proba.to_dict()).fillna(trap_wnv_proba.median())

In [25]:
from sklearn.preprocessing import KBinsDiscretizer


lat_enc = KBinsDiscretizer(n_bins=3, encode='ordinal')

train_df['lat_bin'] = lat_enc.fit_transform(train_df[['Latitude']])
test_df['lat_bin'] = lat_enc.transform(test_df[['Latitude']])

long_enc = KBinsDiscretizer(n_bins=3, encode='ordinal')

train_df['long_bin'] = long_enc.fit_transform(train_df[['Longitude']])
test_df['long_bin'] = long_enc.transform(test_df[['Longitude']])

In [26]:
from src.features import SpeciesEncoder

species_enc = SpeciesEncoder()

train_df = species_enc.fit_transform(train_df)
test_df = species_enc.transform(test_df)

In [27]:
train_columns = ['Week', 'Dayofyear', 'Dayofweek',
       'trap_max_mos', 'trap_wnv_proba', 'lat_bin', 'long_bin',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS', 'WnvPresent']

test_columns = ['Id', 'Week', 'Dayofyear', 'Dayofweek',
       'trap_max_mos', 'trap_wnv_proba', 'lat_bin', 'long_bin',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS']

In [28]:
X_train = train_df[train_columns].copy()
y_train = X_train.pop('WnvPresent')

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold


scaler = StandardScaler()
clf = LogisticRegression()

steps = [
    ('scale', scaler),
    ('classifier', clf)
]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

results_cv = cross_validate(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')

print(results_cv['test_score'].mean())

0.7687091132165861


In [32]:
X_train.shape

(7837, 10)

In [32]:
weather_columns = weather_df.drop('Date', axis=1).columns.to_list()
weather_columns

['Tmax_1',
 'Tmin_1',
 'Tavg_1',
 'DewPoint_1',
 'WetBulb_1',
 'PrecipTotal_1',
 'AvgSpeed_1',
 'ResultSpeed_1',
 'ResultDir_1',
 'Tmax_2',
 'Tmin_2',
 'Tavg_2',
 'DewPoint_2',
 'WetBulb_2',
 'PrecipTotal_2',
 'AvgSpeed_2',
 'ResultSpeed_2',
 'ResultDir_2']

In [33]:
train_df = train_df.merge(weather_df, on='Date')
test_df = test_df.merge(weather_df, on='Date')

In [34]:
X_train = train_df[train_columns+weather_columns].copy()
y_train = X_train.pop('WnvPresent')

In [35]:
scaler = StandardScaler()
clf = LogisticRegression()

steps = [
    ('scale', scaler),
    ('classifier', clf)
]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

results_cv = cross_validate(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')

print(results_cv['test_score'].mean())

0.8147198980682312


In [36]:
# from lightgbm import LGBMClassifier


# scaler = StandardScaler()
# clf = LGBMClassifier()

# steps = [
#     ('scale', scaler),
#     ('classifier', clf)
# ]
# pipeline = Pipeline(steps=steps)

# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

# results_cv = cross_validate(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')

# print(results_cv['test_score'].mean())

In [37]:
from sklearn.metrics import roc_auc_score


pipeline.fit(X_train, y_train)
roc_auc_score(y_train, pipeline.predict_proba(X_train)[:,1])

np.float64(0.8249350216296842)

In [38]:

X_test = test_df[test_columns+weather_columns].reset_index(drop=True)
y_pred = pipeline.predict_proba(X_test.drop('Id', axis=1))[:,1]
X_test['pred_proba'] = y_pred


In [39]:
predictions_df = X_test[['Id', 'pred_proba']].copy()
predictions_df

,Id,pred_proba
0,1,0.035544
1,2,0.013578
2,3,0.043321
3,9,0.008039
4,10,0.003018
...,...,...
43632,115071,0.005528
43633,115072,0.017958
43634,115078,0.015098
43635,115079,0.005693


In [40]:
(predictions_df.pred_proba!=0).sum()

np.int64(43637)

In [41]:
sample = pd.read_csv('../data/raw/sampleSubmission.csv')
sample

,Id,WnvPresent
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
116288,116289,0
116289,116290,0
116290,116291,0
116291,116292,0


In [42]:
results = pd.merge(sample, predictions_df, on='Id', how='outer').fillna(0)
results

,Id,WnvPresent,pred_proba
0,1,0,0.035544
1,2,0,0.013578
2,3,0,0.043321
3,4,0,0.000000
4,5,0,0.000000
...,...,...,...
116288,116289,0,0.000000
116289,116290,0,0.000000
116290,116291,0,0.000000
116291,116292,0,0.000000


In [43]:
results['WnvPresent'] = results['pred_proba']
results.drop('pred_proba', axis=1, inplace=True)

In [44]:
(results.WnvPresent!=0).sum()

np.int64(43637)

In [45]:
results.to_csv('../data/submission/test_results.csv', index=False)

In [46]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [47]:
api.competition_submit('../data/submission/test_results.csv', 'test_lr', 'predict-west-nile-virus')

100%|██████████| 1.80M/1.80M [00:03<00:00, 599kB/s] 


Successfully submitted to West Nile Virus Prediction

In [48]:
api.competitions_submissions_list(id='predict-west-nile-virus')[0]['privateScore']

'0.70988'

In [50]:
train_columns

['Week',
 'Dayofyear',
 'Dayofweek',
 'trap_max_mos',
 'trap_wnv_proba',
 'lat_bin',
 'long_bin',
 'Species_CULEX PIPIENS',
 'Species_CULEX PIPIENS/RESTUANS',
 'Species_CULEX RESTUANS',
 'WnvPresent']

In [52]:
weather_columns

['Tmax_1',
 'Tmin_1',
 'Tavg_1',
 'DewPoint_1',
 'WetBulb_1',
 'PrecipTotal_1',
 'AvgSpeed_1',
 'ResultSpeed_1',
 'ResultDir_1',
 'Tmax_2',
 'Tmin_2',
 'Tavg_2',
 'DewPoint_2',
 'WetBulb_2',
 'PrecipTotal_2',
 'AvgSpeed_2',
 'ResultSpeed_2',
 'ResultDir_2']

In [53]:
column_set_1 = ['trap_max_mos', 'trap_wnv_proba', 'lat_bin', 'long_bin', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
                'Species_CULEX RESTUANS', 'Tmax_1', 'Tmax_2', 'PrecipTotal_1', 'PrecipTotal_2', 'AvgSpeed_1', 'AvgSpeed_2']
column_set_2 = ['Week', 'trap_max_mos', 'trap_wnv_proba', 'lat_bin', 'long_bin', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
                'Species_CULEX RESTUANS', 'Tavg_1', 'Tavg_2', 'WetBulb_1', 'WetBulb_2', 'ResultSpeed_1', 'ResultSpeed_2']
column_set_3 = ['trap_max_mos', 'trap_wnv_proba', 'lat_bin', 'long_bin', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
                'Species_CULEX RESTUANS'] + weather_columns

In [59]:
columnn_sets = [column_set_1, column_set_2, column_set_3]

predictions_df = test_df[['Id']].reset_index(drop=True)

for i in range(len(columnn_sets)):
    
    X_train = train_df[columnn_sets[i]].copy()
    y_train = train_df['WnvPresent'].copy()
    
    scaler = StandardScaler()
    clf = LogisticRegression()

    steps = [
        ('scale', scaler),
        ('classifier', clf)
    ]
    pipeline = Pipeline(steps=steps)

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

    results_cv = cross_validate(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')

    print(results_cv['test_score'].mean())
    
    pipeline.fit(X_train, y_train)
    train_score = roc_auc_score(y_train, pipeline.predict_proba(X_train)[:,1])
    print(train_score)
    
    X_test = test_df[columnn_sets[i]].reset_index(drop=True)
    y_pred = pipeline.predict_proba(X_test)[:,1]
    predictions_df[f'pred_proba_{i}'] = y_pred

0.7707864022183589
0.776970117392767
0.7751399111365753
0.7817983154748259
0.7946670593581767
0.8065698863585615


In [63]:
predictions_df['pred_proba'] = predictions_df[['pred_proba_0', 'pred_proba_1','pred_proba_2']].mean(axis=1)
predictions_df

,Id,pred_proba_0,pred_proba_1,pred_proba_2,pred_proba
0,1,0.133959,0.087829,0.121334,0.114374
1,2,0.043516,0.027758,0.041510,0.037595
2,3,0.188991,0.125566,0.160459,0.158339
3,9,0.035216,0.021330,0.030762,0.029102
4,10,0.010622,0.006421,0.009856,0.008966
...,...,...,...,...,...
43632,115071,0.007542,0.010884,0.001880,0.006769
43633,115072,0.037467,0.052441,0.008244,0.032717
43634,115078,0.024948,0.034460,0.005956,0.021788
43635,115079,0.007470,0.010472,0.001876,0.006606


In [64]:
results = pd.merge(sample, predictions_df, on='Id', how='outer').fillna(0)
results

,Id,WnvPresent,pred_proba_0,pred_proba_1,pred_proba_2,pred_proba
0,1,0,0.133959,0.087829,0.121334,0.114374
1,2,0,0.043516,0.027758,0.041510,0.037595
2,3,0,0.188991,0.125566,0.160459,0.158339
3,4,0,0.000000,0.000000,0.000000,0.000000
4,5,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
116288,116289,0,0.000000,0.000000,0.000000,0.000000
116289,116290,0,0.000000,0.000000,0.000000,0.000000
116290,116291,0,0.000000,0.000000,0.000000,0.000000
116291,116292,0,0.000000,0.000000,0.000000,0.000000


In [65]:
results['WnvPresent'] = results['pred_proba']
results.drop('pred_proba', axis=1, inplace=True)

In [70]:
results = 

,Id,WnvPresent,pred_proba_0,pred_proba_1,pred_proba_2
0,1,0.114374,0.133959,0.087829,0.121334
1,2,0.037595,0.043516,0.027758,0.041510
2,3,0.158339,0.188991,0.125566,0.160459
3,4,0.000000,0.000000,0.000000,0.000000
4,5,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
116288,116289,0.000000,0.000000,0.000000,0.000000
116289,116290,0.000000,0.000000,0.000000,0.000000
116290,116291,0.000000,0.000000,0.000000,0.000000
116291,116292,0.000000,0.000000,0.000000,0.000000


In [71]:
results[['Id', 'WnvPresent']].to_csv('../data/submission/test_results.csv', index=False)

In [72]:
api.competition_submit('../data/submission/test_results.csv', 'test_3lr', 'predict-west-nile-virus')

100%|██████████| 1.80M/1.80M [00:01<00:00, 1.15MB/s]


Successfully submitted to West Nile Virus Prediction

In [73]:
api.competitions_submissions_list(id='predict-west-nile-virus')[0]['privateScore']

'0.67276'